# Define Helper Function

In [1]:
def find_max(col):
    max_num = -999
    for i in range(106):
        if col[i] > max_num:
            max_num = col[i]
    return max_num

def find_min(col):
    min_num = 100000
    for i in range(106):
        if col[i] < min_num:
            min_num = col[i]
    return min_num

def get_goodness(chosen_subset): # (1) 取得要算 goodness 的那些 attribute subset
    # (2) 計算 goodness 的分子，表示 relevant
    relevant_score = 0
    for attr_num in chosen_subset:
        relevant_score = relevant_score + Symmetric_Uncertainty[attr_num][9]

    # (3) 計算 goodness 的分母，表示 duplicate
    duplicate_score = 0
    for attr_num_A in chosen_subset:
        for attr_num_B in chosen_subset:
            duplicate_score = duplicate_score + Symmetric_Uncertainty[attr_num_A][attr_num_B]
    duplicate_score = math.sqrt(duplicate_score)
    goodness = relevant_score/duplicate_score
    return goodness

def argmax(lst): 
    return lst.index(max(lst))

def sort_col(arr):
    
    # 先讓 arr_sort 裡面全部都是 -100
    arr_sort = []
    for i in range(106):
        arr_sort.append(-100)
        
    for count in range(106):

        # 找到當前 arr 裡面的最大值 以及 他的 index
        max_number = -1000
        max_number_idx = -1000
        for i in range(106):

            if arr[i] > max_number:
                max_number = arr[i]
                max_number_idx = i

        # 把他便最小，這樣之後就不會挑到他了
        arr[max_number_idx] = -1000

        # 把當前最大值存到 arr_sort 裡面
        arr_sort[count] = max_number
    
    return arr_sort

# Read Txt File

In [2]:
f = open("../Breast.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9 = [], [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(float(row_list[8]))
    attr_9.append(row_list[9])
    

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有106個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9]

# (EWD 01) Equal-Width Discretization

In [3]:
Attributes_EWD = []
for i in range(10):
    
    specific_col = Attributes[i]
    
    if i != 9:
        number_of_intervals = 10
        max_num_in_col = find_max(specific_col)
        min_num_in_col = find_min(specific_col)
        width = (max_num_in_col - min_num_in_col)/number_of_intervals

        # Decide 9 Split points, so that 10 intervals will form
        split_point_1 = max_num_in_col - 1 * width
        split_point_2 = max_num_in_col - 2 * width
        split_point_3 = max_num_in_col - 3 * width
        split_point_4 = max_num_in_col - 4 * width
        split_point_5 = max_num_in_col - 5 * width
        split_point_6 = max_num_in_col - 6 * width
        split_point_7 = max_num_in_col - 7 * width
        split_point_8 = max_num_in_col - 8 * width
        split_point_9 = max_num_in_col - 9 * width

        # transfer attributes into proper interval
        discrete_attr = []
        for i in range(106):
            discrete_attr.append(-100)

        for i in range(106):

            num = specific_col[i]

            if num >= split_point_1: discrete_attr[i] = 0
            elif num >= split_point_2 and num < split_point_1: discrete_attr[i] = 1
            elif num >= split_point_3 and num < split_point_2: discrete_attr[i] = 2
            elif num >= split_point_4 and num < split_point_3: discrete_attr[i] = 3
            elif num >= split_point_5 and num < split_point_4: discrete_attr[i] = 4
            elif num >= split_point_6 and num < split_point_5: discrete_attr[i] = 5
            elif num >= split_point_7 and num < split_point_6: discrete_attr[i] = 6
            elif num >= split_point_8 and num < split_point_7: discrete_attr[i] = 7
            elif num >= split_point_9 and num < split_point_8: discrete_attr[i] = 8
            elif num < split_point_9: discrete_attr[i] = 9
                
        Attributes_EWD.append(discrete_attr)
    else:
        Attributes_EWD.append(specific_col)

# (EWD 02) Derive Entropy For Each Column

In [4]:
import math

Entropy = []

for attr in Attributes_EWD:
    ##########################
    # (1) 找到 Unique 值有哪些 #
    ##########################
    unique_list = []

    for element in attr:
        if element not in unique_list:
            unique_list.append(element)
        else:
            pass

    # 觀察有哪些 unique 值
    print(f"The unique values are {unique_list}")

    ######################################
    # (2) 針對 Unique 值做計數(Count)的動作 #
    ######################################
    unique_prob_list = []

    # 掃過所有的 unique 值
    for unique_val in unique_list:

        # 計數歸零
        tmp_count = 0

        # 一個一個檢查
        for element in attr:

            # 如果檢查到一致，那就加一
            if element == unique_val: 
                tmp_count = tmp_count + 1
            else:
                pass

        # 把個數換成機率存起來，確認此資料及的資料個數都有106個(i.e., DATA_COUNT)
        unique_prob_list.append(tmp_count/DATA_COUNT)

    # 觀察其機率分佈情況
    print(f"The unique probabilities are {unique_prob_list}")

    ########################
    # (3) 算這個值的 Entropy #
    ########################

    entropy = 0
    for unique_proba in unique_prob_list:
        entropy = entropy - unique_proba * math.log(unique_proba, 2) # Entropy 是以2為底
    print(f"The Entropy is {entropy}\n")
    
    Entropy.append(entropy)
    
# 我只要透過 Entropy[0] 就能取得 第一個 Attribute 的 Entropy

The unique values are [8, 9, 3, 5, 6, 7, 4, 2, 1, 0]
The unique probabilities are [0.16981132075471697, 0.49056603773584906, 0.07547169811320754, 0.018867924528301886, 0.018867924528301886, 0.03773584905660377, 0.08490566037735849, 0.03773584905660377, 0.03773584905660377, 0.02830188679245283]
The Entropy is 2.4188098141439287

The unique values are [4, 3, 6, 5, 1, 2, 0, 8, 9, 7]
The unique probabilities are [0.10377358490566038, 0.0660377358490566, 0.1792452830188679, 0.08490566037735849, 0.009433962264150943, 0.009433962264150943, 0.009433962264150943, 0.27358490566037735, 0.11320754716981132, 0.1509433962264151]
The Entropy is 2.8142772574388983

The unique values are [8, 3, 7, 4, 6, 5, 0, 9, 1, 2]
The unique probabilities are [0.19811320754716982, 0.0660377358490566, 0.32075471698113206, 0.11320754716981132, 0.1509433962264151, 0.07547169811320754, 0.018867924528301886, 0.03773584905660377, 0.009433962264150943, 0.009433962264150943]
The Entropy is 2.7101521815073255

The unique va

# (EWD 03) Derive Joint Entropy for Each two different Columns (Atrributes)

In [5]:
Joint_Entropy = []

for attr_A in Attributes_EWD:
    
    entropy_list_for_A = []
    
    for attr_B in Attributes_EWD:
        
        #################################
        # (1) A column 的 unique 值有哪些 #
        #################################
        unique_list_A = []

        for element in attr_A:
            if element not in unique_list_A:
                unique_list_A.append(element)
            else:
                pass

        # 觀察有哪些 unique 值
        print(f"The unique values of Attribute A are {unique_list_A}")

        #################################
        # (2) B column 的 unique 值有哪些 #
        #################################
        unique_list_B = []

        for element in attr_B:
            if element not in unique_list_B:
                unique_list_B.append(element)
            else:
                pass

        # 觀察有哪些 unique 值
        print(f"The unique values of Attribute B are {unique_list_B}")

        #############################
        # (3) 針對 unique pair 做計數 #
        #############################
        unique_prob_list = []

        # 掃過所有的 unique 值
        for unique_val_A in unique_list_A:
            for unique_val_B in unique_list_B:

                # 計數歸零
                tmp_count = 0

                # 一個一個檢查
                for number in range(DATA_COUNT):
                    
                    element_A = attr_A[number]
                    element_B = attr_B[number]

                    # 如果檢查到一致，那就加一
                    if element_A == unique_val_A and element_B == unique_val_B: 
                        tmp_count = tmp_count + 1
                    else:
                        pass

                # 把個數換成機率存起來，要除的是 106
                unique_prob_list.append(tmp_count/DATA_COUNT)

        # 觀察其機率分佈情況
        print(f"The unique probabilities are {unique_prob_list}")

        #######################################
        # (4) 算這兩個 column (A, B) 的 entropy #
        #######################################
        entropy_A_B = 0
        for unique_proba in unique_prob_list:
            if unique_proba == 0:
                pass
            else:
                entropy_A_B = entropy_A_B - unique_proba * math.log(unique_proba, 2) # Entropy 是以2為底
        print(f"The Entropy is {entropy_A_B}\n")
        
        entropy_list_for_A.append(entropy_A_B)
        
    Joint_Entropy.append(entropy_list_for_A)

# 我只要透過 Joint_Entropy[0][19] 就能取得 第一個 和 第二十個 Attribute 的 Joint Entropy

The unique values of Attribute A are [8, 9, 3, 5, 6, 7, 4, 2, 1, 0]
The unique values of Attribute B are [8, 9, 3, 5, 6, 7, 4, 2, 1, 0]
The unique probabilities are [0.16981132075471697, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.49056603773584906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07547169811320754, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018867924528301886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018867924528301886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03773584905660377, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08490566037735849, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03773584905660377, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03773584905660377, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02830188679245283]
The Entropy is 2.4188098141439287

The unique values of Attribute A are [8, 9, 3, 5, 6, 7, 4, 2, 1, 0]
The unique values of Attribute B are [4, 3, 6, 5, 1, 2, 0, 8, 9, 7]
T

# (EWD 04) Derive Symmetric Uncertainty for Each two different Atrributes

In [6]:
number_of_Attributes = len(Attributes_EWD)
Symmetric_Uncertainty = []

for num_A in range(number_of_Attributes):
    su_for_A = []
    
    for num_B in range(number_of_Attributes):
        if num_A == num_B:
            su = 1
        elif Entropy[num_A]+Entropy[num_B] == 0:
            su = 0
        else:
            su = 2*(Entropy[num_A]+Entropy[num_B]-Joint_Entropy[num_A][num_B])/(Entropy[num_A]+Entropy[num_B])
        su_for_A.append(su)
    
    Symmetric_Uncertainty.append(su_for_A)
    
# 我只要透過 Symmetric_Uncertainty[0][8] 就能取得 第一個 和 第八個 Attribute 的 Symmetric Uncertainty

# (EWD 05) Forward Selection

In [7]:
must_chosen_subset = []
possible_subset = [0,1,2,3,4,5,6,7,8]
best_goodness = 0

while True:
    
    # 一個 goodness 會對應到 一個 experiment 的 col
    goodness_list = []
    exp_attr_list = []

    # 進行同樣 attr 數目的搜索
    for attr_num in possible_subset:

        # 每次搜索過相同 level 的 subset 時，需要清空 chosen_subset
        chosen_subset = [] # zero_subset 的概念

        # 把那些 最棒的 attr 加進來
        for best_attr_num in must_chosen_subset:
            chosen_subset.append(best_attr_num)

        # 這時候從 possible_subset 加一個候選的 attr 進到 chosen_subset
        chosen_subset.append(attr_num)
        exp_attr_list.append(attr_num)

        # 計算 goodness 並且將 goodness 放入 tmp_goodness_list
        goodness = get_goodness(chosen_subset)
        goodness_list.append(goodness)
        print(f"Chosen S: {chosen_subset}; Goodness: {round(goodness,3)}")

    # 找到 goodness 最大的 idx, 並透過 tmp_exp_col 找到 attr_num 和 goodness
    idx = argmax(goodness_list)
    current_best_atrr = exp_attr_list[idx]
    current_best_goodness = goodness_list[idx]

    # 設定停止條件 
    # (1) current_best_goodness 比 best_goodness 低 
    # (2) possible_subset 用完了
    if (current_best_goodness < best_goodness) or (len(possible_subset) < 2):
        break;
    else:
        # 把當前最好的 attr 加進 must_chosen_subset 裡面
        must_chosen_subset.append(current_best_atrr)
        # 把當前最好的 attr 從 possible_subset 裡面取出
        possible_subset.remove(current_best_atrr)
        # 更新歷史上最好的 best_goodness
        best_goodness = current_best_goodness

print()
print(f"Finish. The best choice based on forward selection is ... {must_chosen_subset}, Goodness: {get_goodness(must_chosen_subset)}")

Chosen S: [0]; Goodness: 0.505
Chosen S: [1]; Goodness: 0.299
Chosen S: [2]; Goodness: 0.168
Chosen S: [3]; Goodness: 0.385
Chosen S: [4]; Goodness: 0.124
Chosen S: [5]; Goodness: 0.355
Chosen S: [6]; Goodness: 0.416
Chosen S: [7]; Goodness: 0.344
Chosen S: [8]; Goodness: 0.513
Chosen S: [8, 0]; Goodness: 0.546
Chosen S: [8, 1]; Goodness: 0.514
Chosen S: [8, 2]; Goodness: 0.437
Chosen S: [8, 3]; Goodness: 0.509
Chosen S: [8, 4]; Goodness: 0.401
Chosen S: [8, 5]; Goodness: 0.526
Chosen S: [8, 6]; Goodness: 0.526
Chosen S: [8, 7]; Goodness: 0.486
Chosen S: [8, 0, 1]; Goodness: 0.563
Chosen S: [8, 0, 2]; Goodness: 0.517
Chosen S: [8, 0, 3]; Goodness: 0.548
Chosen S: [8, 0, 4]; Goodness: 0.488
Chosen S: [8, 0, 5]; Goodness: 0.567
Chosen S: [8, 0, 6]; Goodness: 0.557
Chosen S: [8, 0, 7]; Goodness: 0.535
Chosen S: [8, 0, 5, 1]; Goodness: 0.579
Chosen S: [8, 0, 5, 2]; Goodness: 0.544
Chosen S: [8, 0, 5, 3]; Goodness: 0.568
Chosen S: [8, 0, 5, 4]; Goodness: 0.512
Chosen S: [8, 0, 5, 6]; Goodne

# (EWD 06) Backward Selection

In [8]:
must_delete_subset = []
possible_subset = [0,1,2,3,4,5,6,7,8]
best_goodness = 0

while True:
    
    # 一個 goodness 會對應到 一個 experiment 的 col
    goodness_list = []
    exp_attr_list = []

    # 進行同樣 attr 數目的搜索
    for attr_num in possible_subset:

        # 每次搜索過相同 level 的 subset 時，需要填滿 chosen_subset
        chosen_subset = [0,1,2,3,4,5,6,7,8] # all subset 的概念
        # 把一定要拿掉的 attr 給 remove 掉 
        for worst_attr_num in must_delete_subset:
            chosen_subset.remove(worst_attr_num)
        # 把本 Run 要實驗的 attr 拿掉
        chosen_subset.remove(attr_num)
        
        # 紀錄本 run 是拿誰(attr)做實驗
        exp_attr_list.append(attr_num)
        # 計算 goodness 並且將 goodness 放入 tmp_goodness_list
        goodness = get_goodness(chosen_subset)
        goodness_list.append(goodness)
        print(f"Chosen S: {chosen_subset}; Goodness: {round(goodness,3)}")

    # 找到 goodness 最大的 idx, 並透過 tmp_exp_col 找到 attr_num 和 goodness
    idx = argmax(goodness_list)
    current_bad_atrr = exp_attr_list[idx]
    current_best_goodness = goodness_list[idx]

    # 設定停止條件 
    # (1) current_best_goodness 比 best_goodness 低 
    # (2) possible_subset 用完了
    if (current_best_goodness < best_goodness) or (len(possible_subset) < 2):
        break;
    else:
        # 把當前最差的 attr 加進 must_delete_subset 裡面
        must_delete_subset.append(current_bad_atrr)
        # 把當前最好的 attr 從 possible_subset 裡面取出
        possible_subset.remove(current_bad_atrr)
        # 更新歷史上最好的 best_goodness
        best_goodness = current_best_goodness

print()
print(f"Finish. The best choice based on backward selection is ... {possible_subset}, Goodness: {get_goodness(possible_subset)}")

Chosen S: [1, 2, 3, 4, 5, 6, 7, 8]; Goodness: 0.518
Chosen S: [0, 2, 3, 4, 5, 6, 7, 8]; Goodness: 0.53
Chosen S: [0, 1, 3, 4, 5, 6, 7, 8]; Goodness: 0.55
Chosen S: [0, 1, 2, 4, 5, 6, 7, 8]; Goodness: 0.542
Chosen S: [0, 1, 2, 3, 5, 6, 7, 8]; Goodness: 0.564
Chosen S: [0, 1, 2, 3, 4, 6, 7, 8]; Goodness: 0.534
Chosen S: [0, 1, 2, 3, 4, 5, 7, 8]; Goodness: 0.535
Chosen S: [0, 1, 2, 3, 4, 5, 6, 8]; Goodness: 0.546
Chosen S: [0, 1, 2, 3, 4, 5, 6, 7]; Goodness: 0.523
Chosen S: [1, 2, 3, 5, 6, 7, 8]; Goodness: 0.54
Chosen S: [0, 2, 3, 5, 6, 7, 8]; Goodness: 0.553
Chosen S: [0, 1, 3, 5, 6, 7, 8]; Goodness: 0.573
Chosen S: [0, 1, 2, 5, 6, 7, 8]; Goodness: 0.566
Chosen S: [0, 1, 2, 3, 6, 7, 8]; Goodness: 0.553
Chosen S: [0, 1, 2, 3, 5, 7, 8]; Goodness: 0.556
Chosen S: [0, 1, 2, 3, 5, 6, 8]; Goodness: 0.571
Chosen S: [0, 1, 2, 3, 5, 6, 7]; Goodness: 0.545
Chosen S: [1, 3, 5, 6, 7, 8]; Goodness: 0.55
Chosen S: [0, 3, 5, 6, 7, 8]; Goodness: 0.559
Chosen S: [0, 1, 5, 6, 7, 8]; Goodness: 0.579
Chosen

# Read Txt File

In [9]:
f = open("../Breast.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9 = [], [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(float(row_list[8]))
    attr_9.append(row_list[9])
    

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有106個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9]

# (EFD 01) Equal-Frequency Discretization - Part I

In [10]:
Attributes_Sort = []

for i in range(10):
    if i < 9:
        Attributes_Sort.append(sort_col(Attributes[i]))
    else:
        Attributes_Sort.append(Attributes[i])
# 我只要透過 Attributes_Sort[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有106個），而且還有由大排到小

# Read Again

In [11]:
number_of_intervals = 10
number_of_attr_in_interval = 106 / number_of_intervals

f = open("../Breast.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9 = [], [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(float(row_list[8]))
    attr_9.append(row_list[9])
    

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有106個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9]

# (EFD 01) Equal-Frequency Discretization - Part II

In [12]:
Attributes_EFD = []

for j in range(10):
    specific_col_sort = Attributes_Sort[j]
    specific_col = Attributes[j]
    
    if j != 9:

        split_point_1 = specific_col_sort[1 * int(number_of_attr_in_interval)]
        split_point_2 = specific_col_sort[2 * int(number_of_attr_in_interval)]
        split_point_3 = specific_col_sort[3 * int(number_of_attr_in_interval)]
        split_point_4 = specific_col_sort[4 * int(number_of_attr_in_interval)]
        split_point_5 = specific_col_sort[5 * int(number_of_attr_in_interval)]
        split_point_6 = specific_col_sort[6 * int(number_of_attr_in_interval)]
        split_point_7 = specific_col_sort[7 * int(number_of_attr_in_interval)]
        split_point_8 = specific_col_sort[8 * int(number_of_attr_in_interval)]
        split_point_9 = specific_col_sort[9 * int(number_of_attr_in_interval)]

        # transfer attributes into proper interval
        discrete_attr = []
        for i in range(106):
            discrete_attr.append(-100)

        for i in range(106):

            num = specific_col[i]

            if num >= split_point_1: discrete_attr[i] = 0
            elif num >= split_point_2 and num < split_point_1: discrete_attr[i] = 1
            elif num >= split_point_3 and num < split_point_2: discrete_attr[i] = 2
            elif num >= split_point_4 and num < split_point_3: discrete_attr[i] = 3
            elif num >= split_point_5 and num < split_point_4: discrete_attr[i] = 4
            elif num >= split_point_6 and num < split_point_5: discrete_attr[i] = 5
            elif num >= split_point_7 and num < split_point_6: discrete_attr[i] = 6
            elif num >= split_point_8 and num < split_point_7: discrete_attr[i] = 7
            elif num >= split_point_9 and num < split_point_8: discrete_attr[i] = 8
            elif num < split_point_9: discrete_attr[i] = 9
                
        Attributes_EFD.append(discrete_attr)
    else:
        Attributes_EFD.append(specific_col)

# (EFD 02) Derive Entropy For Each Column

In [13]:
import math

Entropy = []

for attr in Attributes_EFD:
    ##########################
    # (1) 找到 Unique 值有哪些 #
    ##########################
    unique_list = []

    for element in attr:
        if element not in unique_list:
            unique_list.append(element)
        else:
            pass

    # 觀察有哪些 unique 值
    print(f"The unique values are {unique_list}")

    ######################################
    # (2) 針對 Unique 值做計數(Count)的動作 #
    ######################################
    unique_prob_list = []

    # 掃過所有的 unique 值
    for unique_val in unique_list:

        # 計數歸零
        tmp_count = 0

        # 一個一個檢查
        for element in attr:

            # 如果檢查到一致，那就加一
            if element == unique_val: 
                tmp_count = tmp_count + 1
            else:
                pass

        # 把個數換成機率存起來，確認此資料及的資料個數都有106個(i.e., DATA_COUNT)
        unique_prob_list.append(tmp_count/DATA_COUNT)

    # 觀察其機率分佈情況
    print(f"The unique probabilities are {unique_prob_list}")

    ########################
    # (3) 算這個值的 Entropy #
    ########################

    entropy = 0
    for unique_proba in unique_prob_list:
        entropy = entropy - unique_proba * math.log(unique_proba, 2) # Entropy 是以2為底
    print(f"The Entropy is {entropy}\n")
    
    Entropy.append(entropy)
    
# 我只要透過 Entropy[0] 就能取得 第一個 Attribute 的 Entropy

The unique values are [3, 6, 5, 7, 4, 8, 9, 1, 2, 0]
The unique probabilities are [0.10377358490566038, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.08490566037735849, 0.09433962264150944, 0.14150943396226415, 0.09433962264150944, 0.09433962264150944, 0.10377358490566038]
The Entropy is 3.3075821031446067

The unique values are [1, 0, 2, 3, 8, 9, 4, 7, 5, 6]
The unique probabilities are [0.09433962264150944, 0.10377358490566038, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.14150943396226415, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944]
The Entropy is 3.308945413712945

The unique values are [8, 0, 6, 1, 4, 7, 2, 3, 5, 9]
The unique probabilities are [0.09433962264150944, 0.10377358490566038, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.14150943396226415]
The Entropy is 3.308945413712945

The unique values 

# (EFD 03) Derive Joint Entropy for Each two different Columns (Atrributes)

In [14]:
Joint_Entropy = []

for attr_A in Attributes_EFD:
    
    entropy_list_for_A = []
    
    for attr_B in Attributes_EFD:
        
        #################################
        # (1) A column 的 unique 值有哪些 #
        #################################
        unique_list_A = []

        for element in attr_A:
            if element not in unique_list_A:
                unique_list_A.append(element)
            else:
                pass

        # 觀察有哪些 unique 值
        print(f"The unique values of Attribute A are {unique_list_A}")

        #################################
        # (2) B column 的 unique 值有哪些 #
        #################################
        unique_list_B = []

        for element in attr_B:
            if element not in unique_list_B:
                unique_list_B.append(element)
            else:
                pass

        # 觀察有哪些 unique 值
        print(f"The unique values of Attribute B are {unique_list_B}")

        #############################
        # (3) 針對 unique pair 做計數 #
        #############################
        unique_prob_list = []

        # 掃過所有的 unique 值
        for unique_val_A in unique_list_A:
            for unique_val_B in unique_list_B:

                # 計數歸零
                tmp_count = 0

                # 一個一個檢查
                for number in range(DATA_COUNT):
                    
                    element_A = attr_A[number]
                    element_B = attr_B[number]

                    # 如果檢查到一致，那就加一
                    if element_A == unique_val_A and element_B == unique_val_B: 
                        tmp_count = tmp_count + 1
                    else:
                        pass

                # 把個數換成機率存起來，要除的是 106
                unique_prob_list.append(tmp_count/DATA_COUNT)

        # 觀察其機率分佈情況
        print(f"The unique probabilities are {unique_prob_list}")

        #######################################
        # (4) 算這兩個 column (A, B) 的 entropy #
        #######################################
        entropy_A_B = 0
        for unique_proba in unique_prob_list:
            if unique_proba == 0:
                pass
            else:
                entropy_A_B = entropy_A_B - unique_proba * math.log(unique_proba, 2) # Entropy 是以2為底
        print(f"The Entropy is {entropy_A_B}\n")
        
        entropy_list_for_A.append(entropy_A_B)
        
    Joint_Entropy.append(entropy_list_for_A)

# 我只要透過 Joint_Entropy[0][19] 就能取得 第一個 和 第二十個 Attribute 的 Joint Entropy

The unique values of Attribute A are [3, 6, 5, 7, 4, 8, 9, 1, 2, 0]
The unique values of Attribute B are [3, 6, 5, 7, 4, 8, 9, 1, 2, 0]
The unique probabilities are [0.10377358490566038, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08490566037735849, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14150943396226415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09433962264150944, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10377358490566038]
The Entropy is 3.3075821031446067

The unique values of Attribute A are [3, 6, 5, 7, 4, 8, 9, 1, 2, 0]
The unique values of Attribute B are [1, 0, 2, 3, 8, 9, 4, 7, 5, 6]
The

# (EFD 04) Derive Symmetric Uncertainty for Each two different Atrributes

In [15]:
number_of_Attributes = len(Attributes_EFD)
Symmetric_Uncertainty = []

for num_A in range(number_of_Attributes):
    su_for_A = []
    
    for num_B in range(number_of_Attributes):
        if num_A == num_B:
            su = 1
        elif Entropy[num_A]+Entropy[num_B] == 0:
            su = 0
        else:
            su = 2*(Entropy[num_A]+Entropy[num_B]-Joint_Entropy[num_A][num_B])/(Entropy[num_A]+Entropy[num_B])
        su_for_A.append(su)
    
    Symmetric_Uncertainty.append(su_for_A)
    
# 我只要透過 Symmetric_Uncertainty[0][8] 就能取得 第一個 和 第八個 Attribute 的 Symmetric Uncertainty

# (EFD 05) Forward Selection

In [16]:
must_chosen_subset = []
possible_subset = [0,1,2,3,4,5,6,7,8]
best_goodness = 0

while True:
    
    # 一個 goodness 會對應到 一個 experiment 的 col
    goodness_list = []
    exp_attr_list = []

    # 進行同樣 attr 數目的搜索
    for attr_num in possible_subset:

        # 每次搜索過相同 level 的 subset 時，需要清空 chosen_subset
        chosen_subset = [] # zero_subset 的概念

        # 把那些 最棒的 attr 加進來
        for best_attr_num in must_chosen_subset:
            chosen_subset.append(best_attr_num)

        # 這時候從 possible_subset 加一個候選的 attr 進到 chosen_subset
        chosen_subset.append(attr_num)
        exp_attr_list.append(attr_num)

        # 計算 goodness 並且將 goodness 放入 tmp_goodness_list
        goodness = get_goodness(chosen_subset)
        goodness_list.append(goodness)
        print(f"Chosen S: {chosen_subset}; Goodness: {round(goodness,3)}")

    # 找到 goodness 最大的 idx, 並透過 tmp_exp_col 找到 attr_num 和 goodness
    idx = argmax(goodness_list)
    current_best_atrr = exp_attr_list[idx]
    current_best_goodness = goodness_list[idx]

    # 設定停止條件 
    # (1) current_best_goodness 比 best_goodness 低 
    # (2) possible_subset 用完了
    if (current_best_goodness < best_goodness) or (len(possible_subset) < 2):
        break;
    else:
        # 把當前最好的 attr 加進 must_chosen_subset 裡面
        must_chosen_subset.append(current_best_atrr)
        # 把當前最好的 attr 從 possible_subset 裡面取出
        possible_subset.remove(current_best_atrr)
        # 更新歷史上最好的 best_goodness
        best_goodness = current_best_goodness

print()
print(f"Finish. The best choice based on forward selection is ... {must_chosen_subset}, Goodness: {get_goodness(must_chosen_subset)}")

Chosen S: [0]; Goodness: 0.457
Chosen S: [1]; Goodness: 0.31
Chosen S: [2]; Goodness: 0.188
Chosen S: [3]; Goodness: 0.41
Chosen S: [4]; Goodness: 0.35
Chosen S: [5]; Goodness: 0.323
Chosen S: [6]; Goodness: 0.42
Chosen S: [7]; Goodness: 0.367
Chosen S: [8]; Goodness: 0.508
Chosen S: [8, 0]; Goodness: 0.529
Chosen S: [8, 1]; Goodness: 0.509
Chosen S: [8, 2]; Goodness: 0.445
Chosen S: [8, 3]; Goodness: 0.533
Chosen S: [8, 4]; Goodness: 0.506
Chosen S: [8, 5]; Goodness: 0.505
Chosen S: [8, 6]; Goodness: 0.546
Chosen S: [8, 7]; Goodness: 0.515
Chosen S: [8, 6, 0]; Goodness: 0.564
Chosen S: [8, 6, 1]; Goodness: 0.553
Chosen S: [8, 6, 2]; Goodness: 0.508
Chosen S: [8, 6, 3]; Goodness: 0.569
Chosen S: [8, 6, 4]; Goodness: 0.534
Chosen S: [8, 6, 5]; Goodness: 0.536
Chosen S: [8, 6, 7]; Goodness: 0.555
Chosen S: [8, 6, 3, 0]; Goodness: 0.581
Chosen S: [8, 6, 3, 1]; Goodness: 0.573
Chosen S: [8, 6, 3, 2]; Goodness: 0.542
Chosen S: [8, 6, 3, 4]; Goodness: 0.553
Chosen S: [8, 6, 3, 5]; Goodness: 

# (EFD 06) Backward Selection

In [17]:
must_delete_subset = []
possible_subset = [0,1,2,3,4,5,6,7,8]
best_goodness = 0

while True:
    
    # 一個 goodness 會對應到 一個 experiment 的 col
    goodness_list = []
    exp_attr_list = []

    # 進行同樣 attr 數目的搜索
    for attr_num in possible_subset:

        # 每次搜索過相同 level 的 subset 時，需要填滿 chosen_subset
        chosen_subset = [0,1,2,3,4,5,6,7,8] # all subset 的概念
        # 把一定要拿掉的 attr 給 remove 掉 
        for worst_attr_num in must_delete_subset:
            chosen_subset.remove(worst_attr_num)
        # 把本 Run 要實驗的 attr 拿掉
        chosen_subset.remove(attr_num)
        
        # 紀錄本 run 是拿誰(attr)做實驗
        exp_attr_list.append(attr_num)
        # 計算 goodness 並且將 goodness 放入 tmp_goodness_list
        goodness = get_goodness(chosen_subset)
        goodness_list.append(goodness)
        print(f"Chosen S: {chosen_subset}; Goodness: {round(goodness,3)}")

    # 找到 goodness 最大的 idx, 並透過 tmp_exp_col 找到 attr_num 和 goodness
    idx = argmax(goodness_list)
    current_bad_atrr = exp_attr_list[idx]
    current_best_goodness = goodness_list[idx]

    # 設定停止條件 
    # (1) current_best_goodness 比 best_goodness 低 
    # (2) possible_subset 用完了
    if (current_best_goodness < best_goodness) or (len(possible_subset) < 2):
        break;
    else:
        # 把當前最差的 attr 加進 must_delete_subset 裡面
        must_delete_subset.append(current_bad_atrr)
        # 把當前最好的 attr 從 possible_subset 裡面取出
        possible_subset.remove(current_bad_atrr)
        # 更新歷史上最好的 best_goodness
        best_goodness = current_best_goodness

print()
print(f"Finish. The best choice based on backward selection is ... {possible_subset}, Goodness: {get_goodness(possible_subset)}")

Chosen S: [1, 2, 3, 4, 5, 6, 7, 8]; Goodness: 0.546
Chosen S: [0, 2, 3, 4, 5, 6, 7, 8]; Goodness: 0.556
Chosen S: [0, 1, 3, 4, 5, 6, 7, 8]; Goodness: 0.57
Chosen S: [0, 1, 2, 4, 5, 6, 7, 8]; Goodness: 0.557
Chosen S: [0, 1, 2, 3, 5, 6, 7, 8]; Goodness: 0.567
Chosen S: [0, 1, 2, 3, 4, 6, 7, 8]; Goodness: 0.563
Chosen S: [0, 1, 2, 3, 4, 5, 7, 8]; Goodness: 0.548
Chosen S: [0, 1, 2, 3, 4, 5, 6, 8]; Goodness: 0.561
Chosen S: [0, 1, 2, 3, 4, 5, 6, 7]; Goodness: 0.539
Chosen S: [1, 3, 4, 5, 6, 7, 8]; Goodness: 0.555
Chosen S: [0, 3, 4, 5, 6, 7, 8]; Goodness: 0.563
Chosen S: [0, 1, 4, 5, 6, 7, 8]; Goodness: 0.568
Chosen S: [0, 1, 3, 5, 6, 7, 8]; Goodness: 0.578
Chosen S: [0, 1, 3, 4, 6, 7, 8]; Goodness: 0.573
Chosen S: [0, 1, 3, 4, 5, 7, 8]; Goodness: 0.556
Chosen S: [0, 1, 3, 4, 5, 6, 8]; Goodness: 0.572
Chosen S: [0, 1, 3, 4, 5, 6, 7]; Goodness: 0.548
Chosen S: [1, 3, 5, 6, 7, 8]; Goodness: 0.563
Chosen S: [0, 3, 5, 6, 7, 8]; Goodness: 0.572
Chosen S: [0, 1, 5, 6, 7, 8]; Goodness: 0.577
Cho

In [29]:
# Entropy-based 
# https://github.com/yangz27/Entropy-and-MDLP-Based-Discretization-Algorithm/blob/710b266649ab159169cda29d89a46ad3597143ec/entropy_based_discretization.py#L51